In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [2]:
# Load the csv exported in Part I to a DataFrame
vacation_df = pd.read_csv("city_weather_test.csv")
vacation_df.drop(columns="Unnamed: 0", inplace=True)
vacation_df.head()

,Lat,Lng,City,Country,Temperature,Humidity,Cloud cover,Wind speed,Date
0,-23.8650,35.3833,inhambane,mz,18.94,57.0,6.0,7.58,2021-07-23 05:13:31
1,70.4692,-68.5914,clyde river,ca,1.45,98.0,100.0,0.76,2021-07-23 05:10:29
2,-3.8004,102.2655,bengkulu,id,28.69,64.0,88.0,2.43,2021-07-23 05:13:33
3,-6.8205,30.4389,karema,tz,21.66,57.0,33.0,4.73,2021-07-23 05:13:34
4,22.0752,-159.3190,kapaa,us,27.99,81.0,90.0,0.45,2021-07-23 05:09:15


 Humidity Heatmap

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)
# Lat and Lng as locations and Humidity as the weight.Add Heatmap layer to map
locations = vacation_df[["Lat", "Lng"]]
humidity = vacation_df["Humidity"]
map_1 = gmaps.figure(zoom_level=2, center=[20,50])
heatmap_1 = gmaps.heatmap_layer(locations, weights=humidity, max_intensity=350, point_radius=6, dissipating=False)
map_1.add_layer(heatmap_1)
map_1

Figure(layout=FigureLayout(height='420px'))

In [4]:
#  Create new DataFrame fitting weather criteria
ideal_vacation = vacation_df[(vacation_df["Temperature"]>=25) & (vacation_df["Temperature"]<=40)]
ideal_vacation = ideal_vacation[ideal_vacation["Wind speed"]<10]
ideal_vacation = ideal_vacation[ideal_vacation["Cloud cover"]<20]
ideal_vacation.head()

,Lat,Lng,City,Country,Temperature,Humidity,Cloud cover,Wind speed,Date
17,40.5971,70.9607,mangit,uz,32.60,17.0,0.0,2.25,2021-07-23 05:13:43
24,40.4272,71.7189,komsomolskiy,ru,31.68,27.0,0.0,2.06,2021-07-23 05:13:49
33,26.9167,87.9167,ilam,ir,26.23,80.0,16.0,2.50,2021-07-23 05:13:57
38,33.6668,-95.5836,lamar,us,26.81,88.0,1.0,2.57,2021-07-23 05:14:01
63,-20.7833,117.1333,roebourne,au,29.59,20.0,12.0,4.12,2021-07-23 05:14:24


In [13]:
# Hotel Map
# Store into variable named hotel_df.Add a "Hotel Name" column to the DataFrame.
hotel_df = ideal_vacation
hotel_df["Hotel Name"] = ""
# Set parameters to search for hotels with 5000 meters.
# Store the first Hotel result into the DataFrame.
# Plot markers on top of the heatmap.

url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

param = {
    "key":g_key,
    "radius":5000,
    "type":"lodging",
    "keyword": "hotel"
}
# param["location"] = f"{lat},{lng}"
ret = requests.get(url, params=param).json()

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    param["location"] = f"{lat},{lng}"
    ret = requests.get(url, params=param).json()

    try:
        hotel_df.loc[index, "Hotel Name"] = ret["results"][0]["name"]
    
    except (KeyError, IndexError):
        print("Missing results")




Missing results
Missing results
Missing results
Missing results
Missing results
Missing results
Missing results
Missing results
Missing results
Missing results


In [17]:
hotel_df.replace('', np.nan, inplace=True)
hotel_df.dropna(inplace=True)
# Save dataframe
hotel_df.to_csv("hotel_df.csv", index=False)

In [22]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations2 = hotel_df[["Lat", "Lng"]]

In [23]:
hotel_df.head()

,Lat,Lng,City,Country,Temperature,Humidity,Cloud cover,Wind speed,Date,Hotel Name
17,40.5971,70.9607,mangit,uz,32.60,17.0,0.0,2.25,2021-07-23 05:13:43,Dangara Hotel
24,40.4272,71.7189,komsomolskiy,ru,31.68,27.0,0.0,2.06,2021-07-23 05:13:49,TANTANA HOTEL FERGANA
33,26.9167,87.9167,ilam,ir,26.23,80.0,16.0,2.50,2021-07-23 05:13:57,Summit Hotel
38,33.6668,-95.5836,lamar,us,26.81,88.0,1.0,2.57,2021-07-23 05:14:01,Days Inn by Wyndham Paris
63,-20.7833,117.1333,roebourne,au,29.59,20.0,12.0,4.12,2021-07-23 05:14:24,Latitude20 Roebourne Village


In [26]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations=locations2, info_box_content=hotel_info)
map_1.add_layer(markers)
# list_of_info = ["""Name]

# Display figure
map_1

Figure(layout=FigureLayout(height='420px'))